In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [3]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
744,(2022/05/31) 23-24,20.05,155
745,Min,15.01,5
746,Max,275,384.1
747,Average,52.019596774193548387096774194,139.58521505376344086021505376


In [4]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
740,(2022/05/31) 19-20,199.24,114.0
741,(2022/05/31) 20-21,85.01,56.5
742,(2022/05/31) 21-22,58.20,72.0
743,(2022/05/31) 22-23,58.22,165.4


In [5]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
740,(2022/05/31) 19-20,199.24,114.0,2022-05-31,19
741,(2022/05/31) 20-21,85.01,56.5,2022-05-31,20
742,(2022/05/31) 21-22,58.20,72.0,2022-05-31,21
743,(2022/05/31) 22-23,58.22,165.4,2022-05-31,22


In [6]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
740,199.24,114.0,2022-05-31,19
741,85.01,56.5,2022-05-31,20
742,58.20,72.0,2022-05-31,21
743,58.22,165.4,2022-05-31,22


In [7]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2022-05-31,199.24,114.0,19,2022,5,31,1
2022-05-31,85.01,56.5,20,2022,5,31,1
2022-05-31,58.20,72.0,21,2022,5,31,1


In [8]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2022 5     27   5331.8  336967.434        63.199564     199.93      25.17   
           28   3602.0  125154.800        34.745919      74.21      25.23   
           29   3144.8   84900.292        26.997040      39.90      25.22   
           30   3757.5  259161.160        68.971699     199.28      25.25   
           31   2238.7  151206.387        67.542050     199.26      20.05   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2022 5     27    63.419583      339.1      177.1  222.158333   
           28    34.795417      181.0      100.0  150.083333   
           29    26.976000      262.8       72.0  157.240000   
           30    70.338500      365.0      129.0  187.875000   
           31    69.930000      243.0       56.5  111.935000   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2016 1     1                48.03              60.08         45.8275   
           2                56.05             118.49         50.2275   
           3                56.01             118.47         50.6050   
           4                80.02             177.66         80.0200   
           5               148.40             181.40         80.0500   
...                           ...                ...             ...   
2022 5     27               70.01             199.93         58.2525   
           28               58.23              74.21         25.2525   
           29               25.30              39.90         25.2450   
           30               70.02             199.28         58.2350   
           31               68.18             199.26         58.1925   

                     date  
year month day             
2016 1     1   2016-01-01  
           2   2016-01-02  
           3   2016-01-03  
           4   2016-01-04  
           5   2016-01-05  
...                   ...  
2022 5     27  2022-05-27  
           28  2022-05-28  
           29  2022-05-29  
           30  2022-05-30  
           31  2022-05-31  

[2343 rows x 13 columns]

In [9]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2022-05-31,199.24,114.0,19,2022,5,31,1,22713.360,evening
2022-05-31,85.01,56.5,20,2022,5,31,1,4803.065,standard
2022-05-31,58.20,72.0,21,2022,5,31,1,4190.400,standard


In [10]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2022 5     27   5331.8  336967.434        63.199564     199.93      25.17   
           28   3602.0  125154.800        34.745919      74.21      25.23   
           29   3144.8   84900.292        26.997040      39.90      25.22   
           30   3757.5  259161.160        68.971699     199.28      25.25   
           31   2238.7  151206.387        67.542050     199.26      20.05   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2022 5     27    63.419583      339.1      177.1  222.158333   
           28    34.795417      181.0      100.0  150.083333   
           29    26.976000      262.8       72.0  157.240000   
           30    70.338500      365.0      129.0  187.875000   
           31    69.930000      243.0       56.5  111.935000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2016 1     1                48.03  ...                 3.0   
           2                56.05  ...                 NaN   
           3                56.01  ...                 NaN   
           4                80.02  ...                 3.0   
           5               148.40  ...                 3.0   
...                           ...  ...                 ...   
2022 5     27               70.01  ...                 3.0   
           28               58.23  ...                 NaN   
           29               25.30  ...                 NaN   
           30               70.02  ...                 3.0   
           31               68.18  ...                 3.0   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2016 1     1                   2.0     3851.5           2920.3   
           2                   NaN     5969.5           1350.0   
           3                   NaN     7198.7              NaN   
           4                   2.0     3828.0            464.0   
           5                   2.0     3230.0            368.4   
...                            ...        ...              ...   
2022 5     27                  2.0     1624.0           2363.0   
           28                  NaN     2507.0           1095.0   
           29                  NaN     3144.8              NaN   
           30                  2.0      777.0           1682.0   
           31                  2.0      735.4            784.5   

                morning_volume  evening_volume   off_value  standard_value  \
year month day                                                               
2016 1     1             993.0           109.0  152306.418      131289.730   
           2               NaN             NaN  271567.021       74415.000   
           3               NaN             NaN  351712.733             NaN   
           4             109.0            56.0  179880.680       32852.590   
           5             109.0            43.0 

In [11]:
daily.to_csv(out + pref + '_daily.csv')

In [12]:
hourly.to_csv(out + pref + '_hourly.csv')

In [13]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2022-05-31,19,199.24,114.0,evening
2022-05-31,20,85.01,56.5,standard
2022-05-31,21,58.20,72.0,standard


In [14]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [15]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,42.0,55.986949,50.402420,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,42.0,67.432548,56.605271,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,46.0,64.633890,55.295628,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,42.0,61.925094,57.116312,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,44.0,56.288962,49.684509,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2022,1,3382.367742,169792.608516,87.711000,26.211667,62.609105,224.009677,73.358065,143.716129,...,42.0,50.199334,29.605810,69.373308,80.235403,258.945786,177.030194,130.076190,114.357895,55.511905
73,2022,2,3102.932143,200037.681286,148.708750,28.682083,94.262009,231.821429,35.146429,129.288839,...,40.0,64.467308,43.553786,88.159573,104.325763,272.262393,172.606481,111.795000,90.604839,44.497500
74,2022,3,2171.461290,169462.048258,170.884167,28.427917,108.949997,164.558065,23.735484,92.065726,...,46.0,78.040557,61.112538,98.337203,105.791747,275.824830,125.226190,88.330435,65.030961,18.930435
75,2022,4,3226.296667,168830.114700,82.855417,26.332917,59.679381,218.770000,58.516667,135.804056,...,42.0,52.329383,29.913547,67.337478,67.743747,118.635343,129.950733,185.846032,138.530451,93.292857


In [16]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2022,1,3382.367742,169792.608516,87.711000,26.211667,62.609105,224.009677,73.358065,143.716129,...,69.373308,80.235403,258.945786,177.030194,130.076190,114.357895,55.511905,21,21,31
73,2022,2,3102.932143,200037.681286,148.708750,28.682083,94.262009,231.821429,35.146429,129.288839,...,88.159573,104.325763,272.262393,172.606481,111.795000,90.604839,44.497500,20,20,28
74,2022,3,2171.461290,169462.048258,170.884167,28.427917,108.949997,164.558065,23.735484,92.065726,...,98.337203,105.791747,275.824830,125.226190,88.330435,65.030961,18.930435,23,23,31
75,2022,4,3226.296667,168830.114700,82.855417,26.332917,59.679381,218.770000,58.516667,135.804056,...,67.337478,67.743747,118.635343,129.950733,185.846032,138.530451,93.292857,21,21,30


In [17]:
monthly=monthly.set_index(['year','month'])

In [18]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [19]:
monthly.to_csv(out + pref + '_monthly.csv')

In [20]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [21]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,183341.846829,52.158401,137.751613,35.304793,62.205317,273.229954,48.152074,154.572665,78.018295,...,2665.800000,5.683597e+06,349.000000,273.000000,66.000000,44.000000,205.889570,115.400886,107.467507,60.177510
1,2,183063.822414,60.187270,163.320354,36.327677,70.596354,246.329293,37.230303,134.219558,99.509798,...,2516.885714,5.178091e+06,326.428571,250.571429,60.428571,40.285714,168.329989,108.906037,106.588469,62.617993
2,3,137644.089627,57.786879,149.784793,34.737926,67.773078,187.270046,39.692166,103.531874,94.613364,...,2623.457143,4.266967e+06,343.142857,276.714286,67.285714,44.857143,123.263495,89.574899,92.543912,58.766155
3,4,146016.226252,49.931672,129.950762,28.604476,56.958559,203.232381,51.434762,125.470246,78.941238,...,3633.928571,4.380487e+06,340.428571,265.142857,63.857143,42.571429,137.998226,114.814591,132.208601,85.638769
4,5,131500.168742,49.862039,144.032442,27.966728,59.190187,200.881567,45.010138,110.867792,79.581014,...,2646.714286,4.076505e+06,347.714286,273.571429,66.428571,44.285714,136.172985,87.055404,110.490717,59.578957
5,6,214057.784206,66.253087,161.234833,36.383278,77.417112,269.190556,46.628889,139.203921,114.329500,...,5597.766667,6.421734e+06,333.333333,249.666667,64.500000,64.500000,177.778448,103.403080,125.502374,87.643241
6,7,271957.968382,69.272739,168.801183,38.594355,80.492093,304.971505,58.989247,166.295968,125.498548,...,6480.866667,8.430697e+06,344.000000,256.000000,66.000000,66.000000,214.353014,126.650901,135.118492,97.062966
7,8,300164.724790,66.900762,165.620323,36.001828,78.735378,351.583333,71.529032,192.473002,124.118925,...,7656.066667,9.305106e+06,340.000000,258.000000,67.000000,67.000000,250.967090,149.177571,148.881696,113.629227
8,9,294415.718811,57.949315,140.910556,33.358833,67.969095,378.627222,92.178889,220.605463,96.025944,...,5130.800000,8.832472e+06,340.333333,265.000000,64.000000,42.666667,282.095454,171.800431,168.346396,123.047152
9,10,294516.099134,57.922255,146.149785,34.331559,68.367194,391.295161,89.031720,239.096703,94.375376,...,3777.383333,9.129999e+06,348.500000,273.500000,66.000000,44.000000,306.670075,195.280506,164.627625,85.005183


In [22]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [23]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      38.032551  221.973980   8390.081342
      1      36.871122  225.456633   8363.403913
      2      36.354235  227.124490   8341.238306
      3      36.247653  229.613265   8428.053602
      4      36.346452  230.485714   8457.179447
...                ...         ...           ...
12    19    152.016183   72.165591   8396.907075
      20    118.552581   93.806452   7105.108511
      21     95.249462  155.121505  10852.770075
      22     41.363495  238.690860   8824.547070
      23     38.810000  243.889247   8855.297274

[288 rows x 3 columns]

In [24]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [26]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price      volume         value
year month hour                                      
2016 1     0      48.389643  356.957143  17095.535893
           1      48.296071  360.239286  17210.836107
           2      48.238571  362.228571  17278.547857
           3      48.355714  369.696429  17682.875179
           4      48.556129  358.341935  17206.331742
...                     ...         ...           ...
2022 5     19    162.213548   95.158065  14166.619161
           20     84.688387  103.158065   7064.263645
           21     47.603871  128.083871   5947.517839
           22     43.971935  119.545161   5128.487903
           23     21.028710  128.032258   2745.202774

[1848 rows x 3 columns]

In [27]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [77]:
from statsmodels.tsa.stattools import pacf

In [78]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.737205,0.714832,0.728297,0.804116
2,-0.077035,-0.182986,-0.172130,-0.257598
3,0.213986,0.323914,0.316676,0.270384
4,0.224259,0.170500,0.198561,0.083137
5,0.035265,0.204603,0.164970,0.068011
6,0.115195,0.154293,0.192543,0.093857
7,-0.011968,-0.118054,-0.144100,-0.007469
8,-0.117650,0.040456,0.021242,0.006867
9,-0.284550,-0.330979,-0.356990,-0.331402


In [79]:
pacfs.to_csv(out + pref + '_pacfs.csv')